In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
import statistics as st

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
print('x' in np.arange(5))   #returns False, without Warning

False


In [4]:
boston = datasets.load_boston() 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target)

In [6]:
## Falta implementar Cross Validation 

In [7]:
X= pd.DataFrame(data=boston.data, columns = boston['feature_names']) 
    
Y = pd.DataFrame(data=boston.target)
    
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [8]:
col_select_train= pd.DataFrame()




In [9]:
len(X_train)

379

In [80]:
col_select_train

""


In [83]:
if len(X_train) == 0:
    print('ok')

In [36]:
X=boston.data
X.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [37]:
DF= pd.DataFrame(data=X, columns = boston['feature_names'])

In [38]:
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


## Forward Selection 

In [44]:
def forward_selection(X=DF,y=boston.target,tol=0.001):
    '''
    X Es un data Frame de features 
    y es una columna de etiquetas 
    
    Elije la mejor variable, una a la vez, para un modelo lineal, y va agregando variables. 
    Devuelve las columnas seleccionadas con sus datos, para el train y el test 
    '''
    
    #X= pd.DataFrame(data=X, columns = boston['feature_names']) 
    
    #Y = pd.DataFrame(data=y)
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,random_state=15)
    
    ## Convertirlos en panda data frame: 
    
    col_select_train= pd.DataFrame()
    col_select_test = pd.DataFrame()
    max_R_2 = 0.1
    max_R_2_ant = 0
    
    while np.abs(max_R_2 - max_R_2_ant) > tol and X_train.shape[1] > 0:         
        
        if len(col_select_train) == 0:  # Primera vuelta, crea el data frame para la primera variable 
            for i in range (X_train.shape[1]):
                R_cuadrado= []
                X_train_m= X_train.iloc[:,i][:, np.newaxis]
                X_test_m= X_test.iloc[:,i][:, np.newaxis]
                
                modelo=LinearRegression()
                modelo.fit(X_train_m, y_train)
                
                #Voy agregando todos los R_cuadrados
                R_cuadrado.append(modelo.score(X_test_m, y_test) ) 
                
            #Me quedo con el mejor R_Cuadrado: 
            max_R_2_ant = max_R_2
            max_R_2 = max(R_cuadrado)
            max_index = R_cuadrado.index(max_R_2)
                
            #Agrego la columna perteneciente al mejor R cuadrado a mis variables seleccionadas:
            col_select_train= pd.DataFrame(data=X_train.iloc[:,max_index])
            col_select_test= pd.DataFrame(data=X_test.iloc[:,max_index])
            print('columnas del modelo seleccionado: ', col_select_train.columns)
            
            
            #Eliminar la columna de X_train y X_test
            X_train.drop(X_train.columns[max_index],axis=1,inplace=True)
            X_test.drop(X_test.columns[max_index],axis=1,inplace=True)
            
        else: 
            for i in range (X_train.shape[1]):
                R_cuadrado= []
                
                ## Panda data frame, con column_select_train + la nueva variable a explorar: 
                X_train_m= pd.concat([col_select_train, X_train.iloc[:,i] ], axis=1)
                X_test_m= pd.concat([col_select_test, X_test.iloc[:,i] ], axis=1)

                modelo=LinearRegression()
                modelo.fit(X_train_m, y_train)
                
                #assert sum(X_train_m['CRIM'] == 'Nan') == 0
                
                #print(X_train_m)
                #print(X_train_m.shape)
                #print('estamos en la i:', i)
                #print('coef', modelo.coef_)
                #print('score', modelo.score(X_test_m, y_test))
                
                #Voy agregando todos los R_cuadrados
                R_cuadrado.append(modelo.score(X_test_m, y_test) ) 
                
            #Me quedo con el mejor R_Cuadrado: 
            max_R_2_ant = max_R_2
            max_R_2 = max(R_cuadrado)
            print('R cuadrado seleccionado es de: ', max_R_2)
            max_index = R_cuadrado.index(max_R_2)

            #Agrego la columna perteneciente al mejor R cuadrado a mis variables seleccionadas:
            col_select_train= pd.concat([col_select_train, X_train.iloc[:,max_index] ], axis=1)
            col_select_test= pd.concat([col_select_test, X_test.iloc[:,max_index] ], axis=1)
            print('columnas del modelo seleccionado: ', col_select_train.columns)
            
            #col_select_train.append(X_train[:,max_index])
            #col_select_test.append(X_test[:,max_index])

            ## Elimino la columna elegida de X_train y X_test para que no pueda volver a ser elegida
            X_train.drop(X_train.columns[max_index],axis=1,inplace=True)
            X_test.drop(X_test.columns[max_index],axis=1,inplace=True)

    
    return col_select_train, col_select_test


NameError: name 'DF' is not defined

In [45]:
col_select_train_0, col_select_test = forward_selection(tol=0.000001)

columnas del modelo seleccionado:  Index(['CRIM'], dtype='object')
R cuadrado seleccionado es de:  0.5337761885389227
columnas del modelo seleccionado:  Index(['CRIM', 'ZN'], dtype='object')
R cuadrado seleccionado es de:  0.5301238007421429
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS'], dtype='object')
R cuadrado seleccionado es de:  0.5296154294453219
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS'], dtype='object')
R cuadrado seleccionado es de:  0.5581579077673682
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX'], dtype='object')
R cuadrado seleccionado es de:  0.564492836339777
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM'], dtype='object')
R cuadrado seleccionado es de:  0.6328006357251179
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE'], dtype='object')
R cuadrado seleccionado es de:  0.6328597105363207
columnas del modelo se

In [13]:
col_select_train_0

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
163,1.51902,0.0,19.58,1.0,0.605,8.375,93.9,2.1620,5.0,403.0,14.7,388.45,3.32
228,0.29819,0.0,6.20,0.0,0.504,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92
84,0.05059,0.0,4.49,0.0,0.449,6.389,48.0,4.7794,3.0,247.0,18.5,396.90,9.62
189,0.08370,45.0,3.44,0.0,0.437,7.185,38.9,4.5667,5.0,398.0,15.2,396.90,5.39
322,0.35114,0.0,7.38,0.0,0.493,6.041,49.9,4.7211,5.0,287.0,19.6,396.90,7.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163,24.0,666.0,20.2,396.90,13.44
133,0.32982,0.0,21.89,0.0,0.624,5.822,95.4,2.4699,4.0,437.0,21.2,388.69,15.03
396,5.87205,0.0,18.10,0.0,0.693,6.405,96.0,1.6768,24.0,666.0,20.2,396.90,19.37
245,0.19133,22.0,5.86,0.0,0.431,5.605,70.2,7.9549,7.0,330.0,19.1,389.13,18.46


In [41]:
col_select_train, col_select_test = forward_selection(tol=0.0001)

columnas del modelo seleccionado:  Index(['CRIM'], dtype='object')
R cuadrado seleccionado es de:  0.5337761885389227
columnas del modelo seleccionado:  Index(['CRIM', 'ZN'], dtype='object')
R cuadrado seleccionado es de:  0.5301238007421429
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS'], dtype='object')
R cuadrado seleccionado es de:  0.5296154294453219
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS'], dtype='object')
R cuadrado seleccionado es de:  0.5581579077673682
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX'], dtype='object')
R cuadrado seleccionado es de:  0.564492836339777
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM'], dtype='object')
R cuadrado seleccionado es de:  0.6328006357251179
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE'], dtype='object')
R cuadrado seleccionado es de:  0.6328597105363207
columnas del modelo se

In [43]:
col_select_train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS
163,1.51902,0.0,19.58,1.0,0.605,8.375,93.9,2.1620
228,0.29819,0.0,6.20,0.0,0.504,7.686,17.0,3.3751
84,0.05059,0.0,4.49,0.0,0.449,6.389,48.0,4.7794
189,0.08370,45.0,3.44,0.0,0.437,7.185,38.9,4.5667
322,0.35114,0.0,7.38,0.0,0.493,6.041,49.9,4.7211
...,...,...,...,...,...,...,...,...
375,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163
133,0.32982,0.0,21.89,0.0,0.624,5.822,95.4,2.4699
396,5.87205,0.0,18.10,0.0,0.693,6.405,96.0,1.6768
245,0.19133,22.0,5.86,0.0,0.431,5.605,70.2,7.9549


In [42]:
col_select_train_2, col_select_test = forward_selection(tol=0.001)

columnas del modelo seleccionado:  Index(['CRIM'], dtype='object')
R cuadrado seleccionado es de:  0.5337761885389227
columnas del modelo seleccionado:  Index(['CRIM', 'ZN'], dtype='object')
R cuadrado seleccionado es de:  0.5301238007421429
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS'], dtype='object')
R cuadrado seleccionado es de:  0.5296154294453219
columnas del modelo seleccionado:  Index(['CRIM', 'ZN', 'INDUS', 'CHAS'], dtype='object')


In [17]:
col_select_train_2

,CRIM,ZN,INDUS,CHAS
163,1.51902,0.0,19.58,1.0
228,0.29819,0.0,6.20,0.0
84,0.05059,0.0,4.49,0.0
189,0.08370,45.0,3.44,0.0
322,0.35114,0.0,7.38,0.0
...,...,...,...,...
375,19.60910,0.0,18.10,0.0
133,0.32982,0.0,21.89,0.0
396,5.87205,0.0,18.10,0.0
245,0.19133,22.0,5.86,0.0


In [18]:
col_select_train_3, col_select_test_2 = forward_selection(tol=0.09)

R cuadrado seleccionado es de:  0.5337761885389227


In [19]:
col_select_train_3

,CRIM,ZN
163,1.51902,0.0
228,0.29819,0.0
84,0.05059,0.0
189,0.08370,45.0
322,0.35114,0.0
...,...,...
375,19.60910,0.0
133,0.32982,0.0
396,5.87205,0.0
245,0.19133,22.0
